# Import Earth Engine module for python API

In [11]:
import geemap, ee

# For authentication, Please load this
# first time load will suffice

# ee.Authenticate()

# Initialize the GEE Api with already verified credentials. 
# This works only if you have authenticated the GEE initally
ee.Initialize()

## Define area of interest

In [2]:
# get our Nepal boundary
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME','Nepal')).geometry()

# Planet & NICFI Basemaps for Tropical Forest Monitoring - Tropical Asia
### https://developers.google.com/earth-engine/datasets/catalog/projects_planet-nicfi_assets_basemaps_asia#description

In [3]:
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/asia')

### Note: You must have access to this data from planet portal, else you will recieve an error
### You can sign up and get details from this portal:
https://www.planet.com/nicfi/

# Sample Vegetation indices

In [5]:
def getNDVI(image):
    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['N','R']).rename("NDVI")
    image = image.addBands(ndvi)

    return(image)

## Filter image for desired dates and apply map function

In [14]:
# Filter basemaps by date and get the first image from filtered results
basemap= nicfi.filter(ee.Filter.date('2021-02-01','2021-07-01')).map(getNDVI).first()

In [19]:
color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
pallete = {"min":0, "max":1, 'palette':color}

# Visualization of Imagery

In [20]:
# initialize our map
map1 = geemap.Map()
map1.centerObject(aoi, 8)
map1.addLayer(basemap.clip(aoi), {"bands":["R","G","B"],"min":64,"max":5454,"gamma":1.8}, "mosiac-false-color-planet")
map1.addLayer(basemap.clip(aoi), {"bands":["N","R","G"],"min":64,"max":5454,"gamma":1.8}, "mosiac-planet")
map1.addLayer(basemap.clip(aoi).select('NDVI'), pallete, "NDVI")

map1.addLayerControl()
map1

Map(center=[28.268224502649357, 83.97637443188768], controls=(WidgetControl(options=['position', 'transparent_…